In [31]:
%load_ext autoreload
%autoreload 2

import os
import sys
import openpyxl
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

sys.path.append("src.py")
from src import Utils

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [32]:
data_dir = "./data/datasets"

### Load proteomics data and reshape to data matrix


In [33]:
data_path = "./data_Proteome/proteomics_filtered.txt"
df = pd.read_csv(data_path, sep="\t")

# index
sample_data = df.copy()
sample_data.set_index("ProteinGroups", inplace = True, drop = False)

# sample data
sample_data = sample_data.drop(['ProteinGroups', 'Genes', 'ProteinDescriptions'], axis = 1).T
sample_data.index = sample_data.index.str.replace('ifng', 'ifng_').str.replace('control', 'control_')

# feature metadata
feature_metadata = df[['ProteinGroups', 'Genes', 'ProteinDescriptions']].copy()
feature_metadata.set_index("ProteinGroups", inplace = True, drop = False)

# sample metadata
sample_metadata = pd.DataFrame({"sample" : sample_data.index})
sample_metadata.set_index("sample", inplace = True, drop = False)

print(f"Data shapes: sample_data : {sample_data.shape}, sample_metadata : {sample_metadata.shape}, feature_metadata : {feature_metadata.shape}")

# Ensure alignment of data and metadata
sample_metadata = sample_metadata.loc[sample_data.index]
feature_metadata = feature_metadata.loc[sample_data.columns]

# Save dataframes
sample_data.to_pickle(os.path.join(data_dir, "islets_proteomics_data.pkl"))
sample_metadata.to_pickle(os.path.join(data_dir, "islets_proteomics_sample_metadata.pkl"))
feature_metadata.to_pickle(os.path.join(data_dir, "islets_proteomics_feature_metadata.pkl"))

Data shapes: sample_data : (18, 7285), sample_metadata : (18, 1), feature_metadata : (7285, 3)


In [34]:
# Delete check

sample_data_old = pd.read_pickle("../../MaTh_Jun24/data/datasets/islets_proteomics_data.pkl")
sample_metadata_old = pd.read_pickle("../../MaTh_Jun24/data/datasets/islets_proteomics_sample_metadata.pkl")
feature_metadata_old = pd.read_pickle("../../MaTh_Jun24/data/datasets/islets_proteomics_feature_metadata.pkl")

print(sample_data_old.equals(sample_data),
sample_metadata_old.equals(sample_metadata),
feature_metadata_old.equals(feature_metadata),)

True True True


### Load transcriptomics data and reshape to data matrix


In [40]:
# data_path = "./data_RNAseq/MARINE_ilots_ifng_expression_DeSeq2.xlsx" # old dataset
data_path = "./data_RNAseq/ilots_ifng_expression_DeSeq2_Marine_20220610.xlsx"
df = pd.read_excel(data_path, engine = 'openpyxl')
print(f"Transcriptomics data shape: {df.shape}")

# remove unused columns
mean_ratio_columns = [
    'alpha.ctrl',
    'alpha.IFNg',
    'alpha.log2fc',
    'alpha.p.value', 
    'alpha.p.adj',
    'beta.ctrl',
    'beta.IFNg',
    'beta.log2fc',
    'beta.p.value', 
    'beta.p.adj',
    'delta.ctrl',
    'delta.IFNg',
    'delta.log2fc', 
    'delta.p.value', 
    'delta.p.adj'
    ]
df.drop(mean_ratio_columns, axis = 1, inplace = True)

# Handle mapping of ENSEMBL to uniprot IDs using bioMart. Prefer static version of mapping dict due to possible API changes
mapping_dict_name = "2025-04-23_12_01_01_mmus_ref_dict.pkl"
if not os.path.exists("2025-04-23_12_01_01_mmus_ref_dict.pkl"):
    mapping_dict_name = f"./{pd.Timestamp.now().strftime('%Y-%m-%d_%H_%M_%S')}_mmus_ref_dict.pkl"
    mmus_ref_dict = Utils.map_ensembl_to_uniprot('mmusculus', True)
    pd.DataFrame.from_dict(mmus_ref_dict, orient = 'index', columns = ['uniprot_id']).to_pickle(mapping_dict_name)

# load mmus_ref_dict from pickle file
mmus_ref_dict = pd.read_pickle(mapping_dict_name)['uniprot_id'].to_dict()

df['uniprot_id'] = df['ensembl.id'].map(mmus_ref_dict)
df.set_index('uniprot_id', inplace = True, drop = False)

# Remove rows with missing uniprot ids
df.dropna(subset = ['uniprot_id'], inplace = True)

# deduplicate by aggregation: average or sum per uniprot id.
df = Utils.deduplicate_alphanumeric_dataframe(df, 'sum')

# index
sample_data = df.copy()
sample_data.set_index('uniprot_id', inplace = True, drop = False)

# sample data
sample_data = sample_data.drop(['ensembl.id', 'gene.symbol', 'uniprot_id'], axis = 1).T
sample_data.index = sample_data.index.str.lower()
sample_data = Utils.nan_safe_df_log(sample_data, 2)

# sample metadata
sample_metadata = pd.DataFrame({"sample" : sample_data.index})
sample_metadata.set_index("sample", inplace = True, drop = False)

# feature metadata
feature_metadata = df[['ensembl.id', 'gene.symbol', 'uniprot_id']].copy()
feature_metadata.set_index('uniprot_id', inplace = True, drop = False)

print(f"Data shapes: sample_data : {sample_data.shape}, sample_metadata : {sample_metadata.shape}, feature_metadata : {feature_metadata.shape}")

# Ensure alignment of data and metadata
sample_metadata = sample_metadata.loc[sample_data.index]
feature_metadata = feature_metadata.loc[sample_data.columns]

# Save dataframes
sample_data.to_pickle(os.path.join(data_dir, "islets_transcriptomics_data.pkl"))
sample_metadata.to_pickle(os.path.join(data_dir, "islets_transcriptomics_sample_metadata.pkl"))
feature_metadata.to_pickle(os.path.join(data_dir, "islets_transcriptomics_feature_metadata.pkl"))


Transcriptomics data shape: (17230, 35)
Shape before deduplication: (13098, 21), shape after deduplication: (12993, 21), aggregated 105 rows.
Data shapes: sample_data : (18, 12993), sample_metadata : (18, 1), feature_metadata : (12993, 3)


In [37]:
# Delete check

sample_data_old = pd.read_pickle("../../MaTh_Jun24/data/datasets/islets_transcriptomics_data.pkl")
sample_metadata_old = pd.read_pickle("../../MaTh_Jun24/data/datasets/islets_transcriptomics_sample_metadata.pkl")
feature_metadata_old = pd.read_pickle("../../MaTh_Jun24/data/datasets/islets_transcriptomics_feature_metadata.pkl")

print(sample_data_old.equals(sample_data),
sample_metadata_old.equals(sample_metadata),
feature_metadata_old.equals(feature_metadata),)

True True True


### Load phospho data and reshape to data matrix


In [38]:
# load phospho data and reshape to data matrix
data_path = "./data_Phospho/celltype_psite.txt"
df = pd.read_csv(data_path, sep="\t")

# index
sample_data = df.copy()
sample_data.set_index('PTM_collapse_key', inplace = True, drop = False)

# sample data
sample_data = sample_data[[
    '20230426_TIMS01scp_MCT_SA_whi20_AID12_DBS08_Phospho_07_alpha_01_S3-G1_1_2886',
    '20230426_TIMS01scp_MCT_SA_whi20_AID12_DBS08_Phospho_08_alpha_02_S3-H1_1_2887',
    '20230426_TIMS01scp_MCT_SA_whi20_AID12_DBS08_Phospho_09_alpha_03_S3-A2_1_2888',
    '20230426_TIMS01scp_MCT_SA_whi20_AID12_DBS08_Phospho_10_beta_01_S3-B2_1_2889',
    '20230426_TIMS01scp_MCT_SA_whi20_AID12_DBS08_Phospho_11_beta_02_S3-C2_1_2890',
    '20230426_TIMS01scp_MCT_SA_whi20_AID12_DBS08_Phospho_12_beta_03_S3-D2_1_2891',
    '20230426_TIMS01scp_MCT_SA_whi20_AID12_DBS08_Phospho_13_delta_01_S3-E2_1_2883',
    '20230426_TIMS01scp_MCT_SA_whi20_AID12_DBS08_Phospho_14_delta_02_S3-F2_1_2884',
    '20230426_TIMS01scp_MCT_SA_whi20_AID12_DBS08_Phospho_15_delta_03_S3-G2_1_2885',]].T

# Apply nan-safe log2 transformation
sample_data = Utils.nan_safe_df_log(sample_data, 2)

# sample_metadata
sample_metadata = pd.DataFrame({"sample" : sample_data.index})
sample_metadata.set_index("sample", inplace = True, drop = False)
sample_metadata['treat'] = 'control'
sample_metadata['sample'] = sample_metadata.index.str.split('_').str[-5]
sample_metadata['rep'] = sample_metadata.index.str.split('_').str[-4].astype(int).astype(str)
sample_metadata['readout'] = 'phosphoproteomics'
sample_metadata['sample_treat_readout'] = sample_metadata['sample'] + '_' + sample_metadata['treat'] + '_' + sample_metadata['readout']
sample_metadata['sample_treat_rep_readout'] = sample_metadata['sample'] + '_' + sample_metadata['treat'] + '_' + sample_metadata['rep'] + '_' + sample_metadata['readout']

# replace data index with matched metadata sample_treat_rep_readout
sample_data = sample_data.join(sample_metadata['sample_treat_rep_readout'])
sample_data.set_index('sample_treat_rep_readout', inplace = True, drop = True)

# replace sample metadata index with sample_treat_rep_readout
sample_metadata['PTM_collapse_key'] = sample_metadata.index
sample_metadata.set_index('sample_treat_rep_readout', inplace = True, drop = False)

# feature metadata
feature_metadata = df[['R.Condition', 'PG.Genes', 'PG.Organisms', 'PG.ProteinDescriptions',
       'PG.ProteinGroups', 'PG.ProteinNames', 'PG.UniProtIds',
       'PEP.PeptidePosition', 'EG.IsDecoy', 'EG.PrecursorId',
       'EG.PTMPositions..Phospho..STY..', 'EG.ApexRT',
       'EG.PTMProbabilities..Phospho..STY..', 'EG.PTMSites..Phospho..STY..',
       'EG.PTMAssayCandidateScore', 'EG.PTMAssayProbability',
       'EG.PTMLocalizationProbabilities', 'EG.ProteinPTMLocations',
       'EG.NormalizationFactor', 'PTM_0_num', 'PTM_group', 'PTM_collapse_key',
       'PTM_collapse_key_num', 'PTM_localization', 'PTM_0_aa']].copy()
feature_metadata.set_index('PTM_collapse_key', inplace = True, drop = False)

print(f"Data shapes: sample_data : {sample_data.shape}, sample_metadata : {sample_metadata.shape}, feature_metadata : {feature_metadata.shape}")

# Ensure alignment of data and metadata
sample_metadata = sample_metadata.loc[sample_data.index]
feature_metadata = feature_metadata.loc[sample_data.columns]
feature_metadata.index.name = 'PTM_collapse_key'

# Handle datatypes (legacy)
feature_metadata['EG.IsDecoy'] = feature_metadata['EG.IsDecoy'].astype(object)
feature_metadata['PTM_collapse_key_num'] = feature_metadata['PTM_collapse_key_num'].astype(object)
feature_metadata['PTM_localization'] = feature_metadata['PTM_localization'].astype(object)

# Save dataframes
sample_data.to_pickle(os.path.join(data_dir, "islets_phospho_data.pkl"))
sample_metadata.to_pickle(os.path.join(data_dir, "islets_phospho_sample_metadata.pkl"))
feature_metadata.to_pickle(os.path.join(data_dir, "islets_phospho_feature_metadata.pkl"))

Data shapes: sample_data : (9, 7356), sample_metadata : (9, 7), feature_metadata : (7356, 25)


In [39]:
# Delete check
sample_data_old = pd.read_pickle("../../MaTh_Jun24/data/datasets/islets_phospho_data.pkl")
sample_data_old = Utils.nan_safe_df_log(sample_data_old, 2)
sample_metadata_old = pd.read_pickle("../../MaTh_Jun24/data/datasets/islets_phospho_sample_metadata.pkl")
feature_metadata_old = pd.read_pickle("../../MaTh_Jun24/data/datasets/islets_phospho_feature_metadata.pkl")

print(sample_data_old.equals(sample_data),
sample_metadata_old.equals(sample_metadata),
feature_metadata_old.equals(feature_metadata),)

True True True


### Save usable datasets for proteomics and transcriptomics data

In [28]:
data_dir = "./data/datasets"
subframes = ['alpha_control', 'beta_control', 'delta_control', 'alpha_ifng', 'beta_ifng', 'delta_ifng']

# load proteomics data
pdata = pd.read_pickle(os.path.join(data_dir, 'islets_proteomics_data.pkl'))

# load transcriptomics data
tdata = pd.read_pickle(os.path.join(data_dir, "islets_transcriptomics_data.pkl"))

# annotate samples with respective modality
pdata.index = [x + "_proteomics" for x in pdata.index]
tdata.index = [x + "_transcriptomics" for x in tdata.index]

id2pg = Utils.map_pg_to_id(pg_series = pdata.columns, id_series = tdata.columns)

# transfer protein group IDs to transcriptomics data
# tdata_mapped_colnames = tdata.columns.to_series().apply(lambda x: id2pg.get(x, x))
# tdata.columns = tdata_mapped_colnames

# transfer transcriptomics IDs to proteomics data
pg2id = {v: k for k, v in id2pg.items()}
pdata_mapped_colnames = pdata.columns.to_series().apply(lambda x: pg2id.get(x, x))
pdata.columns = pdata_mapped_colnames

# match proteomics and transcriptomics data by feature
data = pdata.T.join(tdata.T, how = 'inner').T
print(data.shape)

# split idx
sample_treat_rep_readout = data.index
sample = sample_treat_rep_readout.str.split("_").str[0]
treat = sample_treat_rep_readout.str.split("_").str[1]
rep = sample_treat_rep_readout.str.split("_").str[2]
readout = sample_treat_rep_readout.str.split("_").str[3]

# make metadata
sample_metadata = pd.DataFrame({
    "sample": sample,
    "treat" : treat,
    "rep": rep,
    "readout": readout,
    "sample_treat_readout": ["_".join(x) for x in zip(sample, treat, readout)],
}, index = data.index)

# make feature metadata
feature_metadata = data.columns.to_frame()
if 'ProteinGroups' in feature_metadata.columns:
    feature_metadata.drop(columns = ['ProteinGroups'], inplace = True)
else:
    feature_metadata.drop(columns = 0, inplace = True)

# Ensure alignment of data and metadata
sample_metadata = sample_metadata.loc[data.index]
feature_metadata = feature_metadata.loc[data.columns]

# Save dataframes
data.to_pickle(os.path.join(data_dir, "islets_rna_prot_dataset.pkl"))
sample_metadata.to_pickle(os.path.join(data_dir, "islets_rna_prot_sample_metadata.pkl"))
feature_metadata.to_pickle(os.path.join(data_dir, "islets_rna_prot_feature_metadata.pkl"))

(36, 5147)


In [29]:
# Delete check

sample_data_old = pd.read_pickle("../../MaTh_Jun24/rna_prot_dataset.pkl")
sample_metadata_old = pd.read_pickle("../../MaTh_Jun24/rna_prot_sample_metadata.pkl")
feature_metadata_old = pd.read_pickle("../../MaTh_Jun24/rna_prot_feature_metadata.pkl")

print(sample_data_old.equals(data),
sample_metadata_old.equals(sample_metadata),
feature_metadata_old.equals(feature_metadata),)


False True False


### Save usable datasets for proteomics and transcriptomics and phosphoproteomics data

In [30]:
# read phospho data
data_dir = "./data/datasets"

# load phosphoproteomics data
ph_data = pd.read_pickle(os.path.join(data_dir, 'islets_phospho_data.pkl'))
ph_sample_metadata = pd.read_pickle(os.path.join(data_dir, 'islets_phospho_sample_metadata.pkl'))
ph_feature_metadata = pd.read_pickle(os.path.join(data_dir, 'islets_phospho_feature_metadata.pkl'))

# load proteomics/transcriptomics data
pt_data = pd.read_pickle(os.path.join(data_dir, 'islets_rna_prot_dataset.pkl'))
pt_sample_metadata = pd.read_pickle(os.path.join(data_dir, 'islets_rna_prot_sample_metadata.pkl'))
pt_feature_metadata = pd.read_pickle(os.path.join(data_dir, 'islets_rna_prot_feature_metadata.pkl'))

# In order to join the data, we must reconcile the phospho-site features and the UniProt proteins:
# Each UniProt ID can have multiple phospho-sites, so the final dataset features are the phospho-sites
ph_data_uniprot_annotated = ph_data.T.join(ph_feature_metadata['PG.UniProtIds'], how = 'inner')

# Join the proteomics/transcriptomics data by UniProt ID
ptp_data = ph_data_uniprot_annotated.merge(pt_data.T, left_on = 'PG.UniProtIds', right_index = True, how = 'inner')

# Extract feature metadata and drop the UniProt ID
ptp_feature_metadata = ptp_data['PG.UniProtIds'].to_frame()
ptp_data.drop(columns = ['PG.UniProtIds'], inplace = True)

# Add the original phospho site feature metadata back in
ptp_feature_metadata = ptp_feature_metadata.join(ph_feature_metadata.drop(columns = {'PG.UniProtIds'}), how = 'left')
ptp_feature_metadata.index.name = ph_feature_metadata.index.name

# Derive sample metadata from column names of ptp_data
ptp_sample_metadata = ptp_data.columns.to_frame()
ptp_sample_metadata.columns = ['sample_treat_rep_readout']

# Add original sample metadata back in 
ptp_sample_metadata_unordered = pd.concat([pt_sample_metadata, ph_sample_metadata.drop(columns = {'sample_treat_rep_readout', 'PTM_collapse_key'})], axis = 0)
ptp_sample_metadata = ptp_sample_metadata_unordered.loc[ptp_sample_metadata.index]

# Ensure alignment of data and metadata
ptp_data = ptp_data.T
ptp_sample_metadata = ptp_sample_metadata.loc[ptp_data.index]
ptp_feature_metadata = ptp_feature_metadata.loc[ptp_data.columns]

# Save dataframes
ptp_data.to_pickle(os.path.join(data_dir, "islets_PTP_dataset.pkl"))
ptp_sample_metadata.to_pickle(os.path.join(data_dir, "islets_PTP_sample_metadata.pkl"))
ptp_feature_metadata.to_pickle(os.path.join(data_dir, "islets_PTP_feature_metadata.pkl"))
